In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.2. Ozone Responder Prediction/Output')
cur_date = "011922"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)

#reading in file
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011822.xlsx", sheet = 2)) %>%
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value)))
#SHOULD MISSING DATA BE NAS OR ZEROS???
demographics_data = data.frame(read_excel("SubjectInfo_011822.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

# Which demographic information and/or soluble mediators in plasma collected at baseline best predict ozone-responder status in asthmatics and non-asthmatics?

In [2]:
plasma_baseline_ozone_data = full_join(demographics_data, ozone_data) %>%
    #only interested in plasma soluble mediators (cytokines/sterols) at baseline
    filter(Sample_Type == "P" & Time_Point == "Pre") %>%
    filter(Category == "Cytokine" | Category == "Sterol") %>%
    select(-Sample_Type, -Time_Point)
head(plasma_baseline_ozone_data)

Joining, by = c("Original_Subject_No", "Subject_No", "Subject_ID", "Disease_Status")



,Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IFNg,8.0557406
2,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IL10,0.3476302
3,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IL12p70,0.4572941
4,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IL13,7.7955008
5,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IL1b,0.2763808
6,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,Cytokine,IL2,0.5728830


Seeing if there are statistical differences in predictors stratified by inflammatory or lung status.

In [3]:
#first pivoting wider to make the tbl_summary function work
wider_data = pivot_wider(plasma_baseline_ozone_data %>%
                         #need to remove to not have duplicate rows in wider df
                         select(-Category), names_from = Variable, values_from = Value)
head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,⋯,Zym,DHL,7DHD,Des,dHLan,Zyme,Lath,7DHC,8DHC,Chol
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,M,W,NH,30,23.3,AS,Fa,⋯,0.057,0.224,0.091,0.641,0.004,3.014,1.969,1.007,1.262,1110.442
7,2,AS_2,F,W,NH,39,32.7,AS,Sp,⋯,0.046,0.106,0.041,0.286,0.015,2.508,1.287,0.376,0.609,673.328
11,3,AS_3,M,W,NH,23,23.0,AS,Fa,⋯,0.046,0.094,0.033,0.279,0.005,1.549,1.200,0.359,0.776,537.509
13,4,AS_4,M,W,NH,20,21.4,AS,Wi,⋯,0.046,0.085,0.047,0.297,0.016,1.833,1.062,0.265,0.558,482.396
15,5,AS_5,F,B,NH,20,34.5,AS,Su,⋯,0.060,0.116,0.050,0.255,0.020,3.628,1.795,0.473,0.414,586.750
16,6,AS_6,F,W,NH,25,25.2,AS,Su,⋯,0.034,0.077,0.022,0.272,0.006,1.914,0.966,0.434,0.495,639.636


These p values are relatively high, so I'm not expecting the model to predict well. 

In [14]:
#summary statistics df
wider_data %>%
  tbl_summary(by = Inflammatory_Responder, missing = "no", 
  include = c('IFNg','IL10','IL12p70','IL13','IL1b','IL2','IL4','IL6','IL8','TNFa','GMCSF','IL12p40',
              'IL15','IL16','IL17','IL1a','IL5','IL7','TNFb','VEGF','Eotaxin','Eotaxin3','IP10','MCP1','MCP4',
              'MDC','MIP1a','MIP1b','TARC','MPO','Utero','25OH_VitD','24OH_Chol','SecoB','25OH_Chol','7Keto_Chol',
              '27OH_Chol','bEP_Chol','Lan','Zym','DHL','7DHD','Des','dHLan','Zyme','Lath','7DHC','8DHC','Chol'), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Utero', p-value omitted:
Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels



**Characteristic**,**N**,"**NR**, N = 8","**R**, N = 16",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
IFNg,24,9.7 (8.9),5.3 (3.1),0.085
IL10,24,0.52 (0.48),0.38 (0.14),0.3
IL12p70,24,0.31 (0.10),0.32 (0.23),0.9
IL13,24,5.97 (1.00),6.84 (3.77),0.5
IL1b,24,NA,NA,0.6
0.14162,NA,7 (88%),15 (94%),NA
0.276380750999999,NA,1 (12%),0 (0%),NA
0.425723693999999,NA,0 (0%),1 (6.2%),NA
IL2,24,0.17 (0.19),0.19 (0.22),0.8


In [15]:
wider_data %>%
  tbl_summary(by = Lung_Responder, missing = "no", 
  include = c('IFNg','IL10','IL12p70','IL13','IL1b','IL2','IL4','IL6','IL8','TNFa','GMCSF','IL12p40',
              'IL15','IL16','IL17','IL1a','IL5','IL7','TNFb','VEGF','Eotaxin','Eotaxin3','IP10','MCP1','MCP4',
              'MDC','MIP1a','MIP1b','TARC','MPO','Utero','25OH_VitD','24OH_Chol','SecoB','25OH_Chol','7Keto_Chol',
              '27OH_Chol','bEP_Chol','Lan','Zym','DHL','7DHD','Des','dHLan','Zyme','Lath','7DHC','8DHC','Chol'), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Utero', p-value omitted:
Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels



**Characteristic**,**N**,"**NR**, N = 11","**R**, N = 13",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
IFNg,24,6.8 (5.9),6.8 (6.1),>0.9
IL10,24,0.34 (0.09),0.50 (0.39),0.2
IL12p70,24,0.30 (0.13),0.33 (0.24),0.7
IL13,24,5.74 (1.60),7.23 (3.93),0.3
IL1b,24,NA,NA,0.7
0.14162,NA,10 (91%),12 (92%),NA
0.276380750999999,NA,1 (9.1%),0 (0%),NA
0.425723693999999,NA,0 (0%),1 (7.7%),NA
IL2,24,0.14 (0.17),0.22 (0.24),0.4


# K Nearest Neighbor

The goal of this analysis is to use cytokine/ sterol data [continuous] to predict inflammatory/lung staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification. It was also chosen since SVM does a better job at handling a large number of predictors, since p < n. 

# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)

In [ ]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification_model_errors = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, 
                                           elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    errors = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
        #extracting accuracy, sens, spec, PPV, NPV to take mean and sd later
        error_values = data.frame(t(c(matrix$overall[1], matrix$byClass[1:4])))

        #adding values to df
        errors = rbind(errors, error_values)
  }
  
  #taking averages/sd 
  errors = errors %>%
    summarise(`Accuracy CV Error` = mean(Accuracy), `Sensitivity CV Error` = mean(Sensitivity), 
              `Specificity CV Error` = mean(Specificity), `PPV CV Error` = mean(Pos.Pred.Value),
              `NPV CV Error` = mean(Neg.Pred.Value), `Accuracy Std Error` = sd(Accuracy),
              `Sensitivity Std Error` = sd(Sensitivity), `Specificity Std Error` = sd(Specificity),
              `PPV CV Error` = sd(Pos.Pred.Value), `NPV CV Error` = sd(Neg.Pred.Value))
  
  return(errors)
}

#calling fn
#linear kernel
svm_linear = svm_classification_model_errors(metabolomics_data, "linear", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial = svm_classification_model_errors(metabolomics_data, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification_model_errors(metabolomics_data, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)

#creating one combined df
final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), final_df[,1:8])
final_df